In [1]:
import os
import sys

import copy
import numpy as np
from astropy.cosmology import Planck18

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd())))
sys.path.append(SCRIPT_DIR)
import gwfast.gwfastGlobals as glob
import gwfast 
from gwfast.waveforms import IMRPhenomD_NRTidalv2
from gwfast.signal import GWSignal
from gwfast.network import DetNet
from fisherTools import CovMatr, compute_localization_region, check_covariance, fixParams
from gwfastUtils import GPSt_to_LMST

TEOBResumS is not installed, only the GWFAST waveform models are available, namely: TaylorF2, IMRPhenomD, IMRPhenomD_NRTidalv2, IMRPhenomHM and IMRPhenomNSBH


In [2]:
print(glob.dirName,SCRIPT_DIR)

/home/rciancarella/.local/lib/python3.10/site-packages /home/rciancarella/DarkSirensStat


In [3]:
keys = ('Mc','eta','dL','theta','phi','iota','psi','tcoal','Phicoal','chi1x','chi2x',
        'chi1y','chi2y','chi1z','chi2z','LambdaTilde','deltaLambda','ecc')
mydata=dict.fromkeys(keys)

In [4]:
print(mydata.keys())

dict_keys(['Mc', 'eta', 'dL', 'theta', 'phi', 'iota', 'psi', 'tcoal', 'Phicoal', 'chi1x', 'chi2x', 'chi1y', 'chi2y', 'chi1z', 'chi2z', 'LambdaTilde', 'deltaLambda', 'ecc'])


In [5]:
alldetectors = copy.deepcopy(glob.detectors)
print('All available detectors are: '+str(list(alldetectors.keys())))

# select only LIGO and Virgo
LVdetectors = {det:alldetectors[det] for det in ['L1', 'H1', 'Virgo']}
print('Using detectors '+str(list(LVdetectors.keys())))

All available detectors are: ['L1', 'H1', 'Virgo', 'KAGRA', 'LIGOI', 'ETS', 'ETMR', 'ETSL', 'ETMRL45d', 'ETMRLpar', 'CE1Id', 'CE2NM', 'CE2NSW']
Using detectors ['L1', 'H1', 'Virgo']


In [6]:
LVdetectors

{'L1': {'lat': 30.563,
  'long': -90.774,
  'xax': 242.71636956358617,
  'shape': 'L'},
 'H1': {'lat': 46.455,
  'long': -119.408,
  'xax': 170.99924234706103,
  'shape': 'L'},
 'Virgo': {'lat': 43.631,
  'long': 10.504,
  'xax': 115.56756342034298,
  'shape': 'L'}}

In [7]:
# We use the O2 psds
LVdetectors['L1']['psd_path'] = os.path.join(glob.detPath, 'LVC_O1O2O3', '2017-08-06_DCH_C02_L1_O2_Sensitivity_strain_asd.txt')
LVdetectors['H1']['psd_path'] = os.path.join(glob.detPath, 'LVC_O1O2O3', '2017-06-10_DCH_C02_H1_O2_Sensitivity_strain_asd.txt')
LVdetectors['Virgo']['psd_path'] = os.path.join(glob.detPath, 'LVC_O1O2O3', 'Hrec_hoft_V1O2Repro2A_16384Hz.txt')


In [8]:
glob.detPath

'/home/rciancarella/.local/lib/python3.10/site-packages/psds'

In [9]:
myLVSignals = {}

for d in LVdetectors.keys():

    myLVSignals[d] = GWSignal(IMRPhenomD_NRTidalv2(),
                psd_path=LVdetectors[d]['psd_path'],
                detector_shape = LVdetectors[d]['shape'],
                det_lat= LVdetectors[d]['lat'],
                det_long=LVdetectors[d]['long'],
                det_xax=LVdetectors[d]['xax'],
                verbose=True,
                useEarthMotion = False,
                fmin=10.,
                IntTablePath=None)

myLVNet = DetNet(myLVSignals)

Using ASD from file /home/rciancarella/.local/lib/python3.10/site-packages/psds/LVC_O1O2O3/2017-08-06_DCH_C02_L1_O2_Sensitivity_strain_asd.txt 
Initializing jax...
Jax local device count: 8
Jax  device count: 8
Using ASD from file /home/rciancarella/.local/lib/python3.10/site-packages/psds/LVC_O1O2O3/2017-06-10_DCH_C02_H1_O2_Sensitivity_strain_asd.txt 
Initializing jax...
Jax local device count: 8
Jax  device count: 8
Using ASD from file /home/rciancarella/.local/lib/python3.10/site-packages/psds/LVC_O1O2O3/Hrec_hoft_V1O2Repro2A_16384Hz.txt 
Initializing jax...
Jax local device count: 8
Jax  device count: 8


In [10]:


# Median values of the posterior samples for all the parameters,
# except psi and the coalescence phase that are set to 0

z = np.array([0.00980])
tGPS = np.array([1187008882.4])

GW170817 = {'Mc':np.array([1.1859])*(1.+z),
            'dL':Planck18.luminosity_distance(z).value/1000.,
            'theta':np.array([np.pi/2. + 0.4080839999999999]),
            'phi':np.array([3.4461599999999994]),
            'iota':np.array([2.545065595974997]),
            'psi':np.array([0.]),
            'tcoal':GPSt_to_LMST(tGPS, lat=0., long=0.), # GMST is LMST computed at long = 0°
            'eta':np.array([0.24786618323504223]),
            'Phicoal':np.array([0.]),
            'chi1z':np.array([0.005136138323169717]),
            'chi2z':np.array([0.003235146993487445]),
            'Lambda1':np.array([368.17802383555687]),
            'Lambda2':np.array([586.5487031450857])
           }

print('Parameters for GW170817 are:')
GW170817

Parameters for GW170817 are:


{'Mc': array([1.19752182]),
 'dL': array([0.04374755]),
 'theta': array([1.97888033]),
 'phi': array([3.44616]),
 'iota': array([2.5450656]),
 'psi': array([0.]),
 'tcoal': DeviceArray([0.43432288], dtype=float64),
 'eta': array([0.24786618]),
 'Phicoal': array([0.]),
 'chi1z': array([0.00513614]),
 'chi2z': array([0.00323515]),
 'Lambda1': array([368.17802384]),
 'Lambda2': array([586.54870315])}

In [11]:
SNR = myLVNet.SNR(GW170817)
print('SNR for GW170817 is %.2f to compare with 33'%SNR[0])

SNR for GW170817 is 33.16 to compare with 33


In [12]:
totF = myLVNet.FisherMatr(GW170817)
print('The computed Fisher matrix has shape %s'%str(totF.shape))

Computing Fisher for L1...
Computing derivatives...
Computing Fisher for H1...
Computing derivatives...
Computing Fisher for Virgo...
Computing derivatives...
Done.
The computed Fisher matrix has shape (13, 13, 1)


In [13]:
totF

array([[[ 2.99788834e+12],
        [-4.34587536e+10],
        [-1.70553758e+04],
        [-2.48002284e+08],
        [-3.82053233e+08],
        [ 3.72516037e+05],
        [-1.16641283e+08],
        [-3.91774476e+10],
        [ 5.73325918e+07],
        [-1.34640982e+10],
        [-1.07415409e+10],
        [-2.12042976e+04],
        [-8.41611419e+01]],

       [[-4.34587536e+10],
        [ 6.49282666e+08],
        [-3.04345233e+03],
        [ 4.10393120e+06],
        [ 6.03306904e+06],
        [-4.80795429e+03],
        [ 1.67779985e+06],
        [ 6.35059458e+08],
        [-8.24678226e+05],
        [ 1.99064378e+08],
        [ 1.58787297e+08],
        [ 3.72479952e+02],
        [ 1.26058035e+00]],

       [[-1.70553758e+04],
        [-3.04345233e+03],
        [ 5.74454126e+05],
        [-6.26631179e+03],
        [-1.14828101e+04],
        [-1.69938852e+04],
        [-9.39098325e+01],
        [ 8.35053518e-01],
        [-2.50113478e-14],
        [-2.20986117e+02],
        [-1.76882522e+02

In [14]:
# Check e.g. that the (dL,dL) element corresponds to (SNR/dL)^2
ParNums = IMRPhenomD_NRTidalv2().ParNums
dL_Num = ParNums['dL']
print('The relative difference is %.2e !'%((1 - totF[ParNums['dL'],ParNums['dL'],:]/(SNR/GW170817['dL'])**2)[0]))

The relative difference is 4.44e-16 !


In [15]:
# Check e.g. that the (dL,dL) element corresponds to (SNR/dL)^2
ParNums = IMRPhenomD_NRTidalv2().ParNums
dL_Num = ParNums['dL']
print('The relative difference is %.2e !'%((1 - totF[ParNums['dL'],ParNums['dL'],:]/(SNR/GW170817['dL'])**2)[0]))

The relative difference is 4.44e-16 !


In [16]:
totCov, inversion_err = CovMatr(totF)

In [17]:
totCov

array([[[ 1.60131987e-07],
        [-2.32468137e-06],
        [-1.10634872e-07],
        [ 6.45159992e-08],
        [ 2.24194551e-08],
        [-5.15201939e-06],
        [ 1.47171559e-05],
        [-3.26982129e-06],
        [-1.04413296e-03],
        [ 1.33517731e-02],
        [-1.65173920e-02],
        [ 1.41914630e-01],
        [-2.02322434e+04]],

       [[-2.32468137e-06],
        [ 2.06089852e-03],
        [ 9.03544641e-06],
        [ 1.13429157e-05],
        [ 8.16331983e-06],
        [-7.43236328e-05],
        [ 2.42845490e-03],
        [-1.74195179e-04],
        [ 1.87873294e-02],
        [ 6.99684852e-01],
        [-8.73327484e-01],
        [ 1.71180500e+01],
        [-1.53308759e+06]],

       [[-1.10634872e-07],
        [ 9.03544641e-06],
        [ 4.89140124e-03],
        [-8.18987655e-05],
        [ 5.11362752e-04],
        [ 1.64988878e-01],
        [-2.93776234e-03],
        [-2.06242358e-06],
        [-1.02537284e-02],
        [-5.90919715e-03],
        [ 7.23119907e-03

In [18]:
_ = check_covariance(totF, totCov)

Inversion errors: [3659.90625]
diagonal-1 = [array([-1.93541609e-08, -1.78260962e-10, -6.41954534e-13, -1.66283654e-13,
        2.13815077e-12, -1.15751159e-12,  1.53178803e-10,  9.27849442e-09,
        4.14024726e-09,  6.40697544e-06,  8.91659874e-06, -1.87899474e-10,
        3.18686944e-09], dtype=float128)]
Max off diagonal: [3659.90625]

mask: where F*S(off-diagonal)>1e-10 (--> problematic if True off diagonal)
[array([[ True,  True, False, False, False, False, False, False, False,
         True, False, False, False],
       [ True,  True, False, False,  True, False, False, False, False,
         True, False, False, False],
       [ True, False,  True, False, False, False, False, False, False,
        False, False, False, False],
       [False,  True, False,  True, False, False, False,  True, False,
         True,  True, False, False],
       [ True, False, False, False,  True, False, False, False, False,
        False, False, False, False],
       [False,  True, False,  True, Fals

'Mc','eta','dL','theta','phi','iota','psi','tcoal','Phicoal','chi1z','chi2z','LambdaTilde','deltaLambda'}

In [23]:
ParNums = IMRPhenomD_NRTidalv2().ParNums

newFish, newPars = fixParams(totF, ParNums, ['deltaLambda'])
print('Now the Fisher matrix has shape %s'%str(newFish.shape))

newCov, new_inversion_err = CovMatr(newFish)

_ = check_covariance(newFish, newCov)

Now the Fisher matrix has shape (12, 12, 1)
Inversion errors: [0.00030831]
diagonal-1 = [array([-4.54829951e-10, -1.78643766e-09, -9.34453795e-13,  1.48501003e-14,
        2.56035319e-13, -8.07555303e-13, -5.37078281e-12,  5.18435364e-10,
       -3.58276742e-11,  7.52511337e-07,  6.57836786e-07, -8.88275564e-12],
      dtype=float128)]
Max off diagonal: [0.00030830537434667348862]

mask: where F*S(off-diagonal)>1e-10 (--> problematic if True off diagonal)
[array([[ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True,  True, False, False, False, False, False,  True, False,
        False,  True, False],
       [False, False,  True, False, False, False, False, False, False,
        False, False, False],
       [False,  True, False,  True, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False,  True, False, False, False, False,
        False, False, False],
       [False,  True, False, False

In [24]:
skyArea = compute_localization_region(newCov, newPars, GW170817['theta'])
print('The estimated sky location is %.1f deg^2, to compare with 16 deg^2'%skyArea)

The estimated sky location is 19.9 deg^2, to compare with 16 deg^2


In [29]:
save_path='/home/rciancarella/testgwfast.h5'
gwfast.gwfastUtils.save_data(save_path, GW170817)
gwfast.gwfastUtils.save_detectors('/home/rciancarella/net.json',LVdetectors)

Saving to /home/rciancarella/testgwfast.h5 


In [22]:
print(newCov)

[[[ 4.93624893e-05]
  [-2.67386259e-05]]

 [[-2.67386259e-05]
  [ 2.64407486e-05]]]


# Play Around

In [ ]:
# Configure ET and the PSD
ETdet = {'ET': copy.deepcopy(glob.detectors).pop('ETS') }
print(ETdet)
ETdet['ET']['psd_path'] = os.path.join(glob.detPath, 'ET-0000A-18.txt')

In [ ]:
from gwfast.waveforms import IMRPhenomD
from gwfast.waveforms import IMRPhenomHM
mySignalsET = {}

for d in ETdet.keys():
    #print(d)
    mySignalsET[d] = GWSignal((IMRPhenomD()),
                psd_path= ETdet[d]['psd_path'],
                detector_shape = ETdet[d]['shape'],
                det_lat= ETdet[d]['lat'],
                det_long=ETdet[d]['long'],
                det_xax=ETdet[d]['xax'],
                verbose=True,
                useEarthMotion = False,
                fmin=2.,
                IntTablePath=None)

myET = DetNet(mySignalsET)

In [ ]:
import pandas as pd
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM

In [ ]:
CAT_FOLDER='/home/rciancarella/DarkSirensStat/DSCatalogueCreator/'
SCRIPT_FOLDER='/home/rciancarella/DarkSirensStat/GWfast/'
os.chdir(CAT_FOLDER)

In [ ]:
dat = Table.read('EVA02_detected12_fiveyear.fits', format='fits')
EVA= dat.to_pandas()#all good, is an only text fits
os.chdir(SCRIPT_FOLDER)

In [ ]:
EVA.head()

Raccordo tra i dati 
GW170817 = {'Mc':np.array([1.1859])*(1.+z),
            'dL':Planck18.luminosity_distance(z).value/1000.,
            'theta':np.array([np.pi/2. + 0.4080839999999999]),
            'phi':np.array([3.4461599999999994]),
            'iota':np.array([2.545065595974997]),
            'psi':np.array([0.]),
            'tcoal':GPSt_to_LMST(tGPS, lat=0., long=0.), # GMST is LMST computed at long = 0°
            'eta':np.array([0.24786618323504223]),
            'Phicoal':np.array([0.]),
            'chi1z':np.array([0.005136138323169717]),
            'chi2z':np.array([0.003235146993487445]),
            'Lambda1':np.array([368.17802383555687]),
            'Lambda2':np.array([586.5487031450857])
           }

In [ ]:
ParNums = IMRPhenomHM().ParNums
print('In totale sono {} parametri da inserire\ne sono {}'.format(len(ParNums),ParNums))

In [ ]:
H0GLOB= 67#67.9 #69
Om0GLOB=0.319
Xi0Glob =1.
cosmoeuclid = FlatLambdaCDM(H0=H0GLOB, Om0=Om0GLOB)

In [ ]:
def chirp_mass(m1,m2):
    num=(m1*m2)**(3/5)
    denom=(m1+m2)**(1/5)
    return num/denom
def phi_from_cart(x,y):
    if(x>0):
        phi=np.arctan(y/x)
    if ((x<0) and (y>=0)):
        phi=np.arctan(y/x)+np.pi
    if ((x<0) and (y<0)):
        phi=np.arctan(y/x)-np.pi
    return phi
def theta_from_cart(x,y,z):
    den=np.sqrt(x**2+y**2+z**2)
    theta=np.arccos(z/den)
    return theta

In [ ]:
allm1=np.asarray(EVA['M1'])
allm2=np.asarray(EVA['M2'])
allMc=chirp_mass(allm1,allm2)
allq=np.asarray(EVA['q'])
alleta=allq/(1+allq**2)
allcos=np.asarray(EVA['cos_orbital_incl'])
alliota=np.arccos(allcos)
allpsi=np.asarray(EVA['psi'])/2
tGPS = np.array([1187008882.4])#arbitrario
allz=np.asarray(EVA['DS_redshift'])
alldsx=np.asarray(EVA['DS_x'])
alldsy=np.asarray(EVA['DS_y'])
alldsz=np.asarray(EVA['DS_z'])
allphi=np.zeros(len(alldsx))
alltheta=np.zeros(len(alldsx))
for i in range(len(alldsx)):
    allphi[i]=phi_from_cart(alldsx[i],alldsy[i])
    alltheta[i]=theta_from_cart(alldsx[i],alldsy[i],alldsz[i])
alldl=cosmoeuclid.luminosity_distance(allz).value/1000.


tcoal=np.asarray(GPSt_to_LMST(tGPS, lat=0., long=0.))

In [ ]:
len(alliota[alliota<0])

In [ ]:
quanti=int(1)
Allevents = {'Mc':1*allMc[0:quanti]*(1+allz)[0:quanti],
            'dL':alldl[0:quanti],
            'theta':alltheta[0:quanti],
            'phi':allphi[0:quanti],
            'iota':alliota[0:quanti],
            'psi':allpsi[0:quanti],
            'tcoal':1*tcoal*np.ones(len(allMc))[0:quanti], # GMST is LMST computed at long = 0°
            'eta':alleta[0:quanti],
            'Phicoal':np.zeros(len(allMc))[0:quanti],
            'chi1z':0.001*np.ones(len(allMc))[0:quanti],
            'chi2z':0.*np.ones(len(allMc))[0:quanti]
            ,#'chi2z':np.zeros(len(allMc))[0:1]
           }

In [ ]:
save_path='/home/rciancarella/DarkSirensStat/GWfast/myfirstevent.h5'
gwfast.gwfastUtils.save_data(save_path, Allevents)

In [ ]:
Allevents

In [ ]:
SNR_ET = myET.SNR(Allevents)
#print('SNR is %.2f '%SNR)
print(SNR_ET)

In [ ]:
totFET = myET.FisherMatr(Allevents)
print('The computed Fisher matrix has shape %s'%str(totFET.shape))

In [ ]:
#print('Fisher Matrix\n{}'.format(totFET))
n_array = totFET[:,:,0]
  
# Displaying the Matrix 
#print("Numpy Matrix is:") 
print(n_array) 
  
# calculating the determinant of matrix 
det = np.linalg.det(n_array) 
  
print("\nDeterminant of given matrix:") 
print(det) 

In [ ]:
totCov_ET, inversion_err_ET = CovMatr(totFET)

In [ ]:
totCov_ET

In [ ]:
os.chdir(SCRIPT_FOLDER+'results/')

In [ ]:
loaded_fish=np.load('fishers_0_to_1.npy')

In [ ]:
np.shape(loaded_fish)